# **Работа с API**

Многие крупные сайты предоставляют доступ к так называемым **API** (англ. Application Programming Interface, рус. Интерфейс Прикладного Программирования). 
 
**API** — это специальные разделы сайта, где информацию можно получать **без разметки**, а **формат запросов и ответов зафиксирован**. API созданы для того, чтобы облегчить взаимодействие с сайтом для сторонних разработчиков.

Рассмотрим на примере социальной сети ВКонтакте особенности API, характерные для более крупных сайтов.

##### **КЛЮЧ АВТОРИЗАЦИИ**
***
Для того чтобы начать работать с API, обычно необходимо получить сервисный ключ авторизации — **токен**.

**Токен** — это средство *идентификации* пользователя или отдельного сеанса работы *в компьютерных сетях и приложениях*. Различают *программные* и *аппаратные* токены.  
Мы будем использовать программный токен, который обычно представляет собой зашифрованную последовательность символов, позволяющую точно идентифицировать объект и определить уровень его привилегий. Он генерируется системой авторизации и привязывается к конкретному сеансу работы, клиенту сети или пакету данных.

Сервисный токен для API ВКонтакте для нашей задачи создаётся вместе с новым приложением. Приложение мы делать, конечно, не будем. Оно нужно только для получения токена, чтобы сделать необходимые выгрузки.

##### **ЗАПРОСЫ ИЗ БРАУЗЕРА**
***
Сначала рассмотрим работу API на простом примере, на основе которого работают многие системы.

Сделаем наш первый запрос из браузера.

Перейдите по следующей ниже ссылке в браузере, подставив вместо слова *TOKEN* ваш персональный сервисный ключ доступа (токен), полученный на предыдущем шаге:

**https://api.vk.com/method/users.get?user_id=1&v=5.95&access_token=TOKEN**

Итак, мы сделали GET-запрос к API ВКонтакте, который состоит из следующих элементов:

* https://api.vk.com/method — домен и URL запроса API; обычно не меняется;
* users.get — название метода, который отдаёт определённый отчёт, в нашем случае это метод для получения информации о пользователе;
* user_id и v — параметры запроса: идентификатор пользователя, о котором хотим получить информацию (в нашем примере мы запрашиваем информацию о первом пользователе), и номер версии API;
* token — токен, который выдаётся только пользователям, имеющим право просматривать определённые данные, например показания счётчиков Яндекс.Метрики вашего проекта; на все остальные запросы без корректного токена система отвечает отказом.

Если мы обратимся к [**документации метода users.get**](https://vk.com/dev/users.get), то увидим, что в ней описано множество других параметров, которые можно получить о пользователе (дата рождения, пол, родной город и другие) — словом, всё то, что мы видим на странице пользователя в интерфейсе или приложении ВКонтакте (конечно, если пользователь их указал).

Добавим к запросу дату рождения и пол (согласно документации, эти параметры надо перечислять в поле fields):

**https://api.vk.com/method/users.get?user_id=1&v=5.95&fields=sex,bdate&access_token=TOKEN**

##### **ЗАПРОС К API ИЗ КОДА**
***
Продолжаем пользоваться всё той же библиотекой requests.

In [26]:
import requests # Импортируем модуль requests
from pprint import pprint
import time # Импортируем модуль time

In [1]:
token = 'dfc65ed2dfc65ed2dfc65ed21fdfba1207ddfc6dfc65ed2bda5c5957a7f5bc7684658c4' # Указываем свой сервисный токен
url = 'https://api.vk.com/method/users.get' # Указываем адрес страницы к которой делаем запрос
params = {'user_id': 1, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': token, 'lang': 'ru'} # Перечисляем параметры нашего запроса в словаре params
response = requests.get(url, params=params) # Отправляем запрос
print(response.text) # Выводим текст ответа на экран

{"response":[{"id":1,"first_name":"Павел","last_name":"Дуров","can_access_closed":true,"is_closed":false,"sex":2,"bdate":"10.10.1984"}]}


Мы получили строку в JSON-формате, которую можно преобразовать в словарь с помощью метода **json()**, после чего можно с лёгкостью обращаться к различным полям.

Словари нагляднее выводить с помощью функции **pprint()**, которую мы уже использовали ранее:

In [3]:
pprint(response.json()) # Выводим содержимое словаря, содержащего ответ, на экран

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}


По ключу response мы можем получить список, в котором хранятся словари, содержащие информацию о запрошенных нами пользователях. Мы запросили информацию лишь об одном из них, поэтому список содержит только один элемент. Извлечём его:

In [4]:
user = response.json()['response'][0] # Извлекаем из словаря по ключу response информацию о первом пользователе
print(user['bdate']) # Выводим дату рождения первого пользователя на экран

10.10.1984


Метод **users.get()** позволяет запрашивать информацию о множестве (до 1 000) пользователей одновременно. Для этого нужно использовать параметр **user_ids** и передавать id **через запятую в строковом формате**. Например, чтобы получить информацию о пользователях с id=1, id=2, id=3, необходимо передать значение параметра user_ids='1,2,3'.

In [5]:
# Формируем строку, содержащую информацию о поле id первых трёх пользователей
ids = ",".join(map(str, range(1, 4)))
# Формируем строку параметров
params = {'user_ids': ids, 'v': 5.95, 'fields': 'bday', 'access_token': token, 'lang': 'ru'}
# Посылаем запрос, полученный ответ в формате JSON-строки преобразуем в словарь
# и выводим на экран его содержимое, используя функцию pprint()
pprint(requests.get(url, params=params).json())

{'response': [{'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'last_name': ''}]}


In [20]:
# Используя API, определите долю женщин (sex=1) среди пользователей с id от 1 до 500. 
# Иногда будут попадаться пользователи, у которых пол не указан (sex=0), — таких пользователей не нужно учитывать в общем числе.
# В ответе укажите число, округлив до двух знаков после точки-разделителя
ids = ",".join(map(str, range(1, 501)))
params = {'user_ids': ids, 'v': 5.95, 'fields': 'sex', 'access_token': token, 'lang': 'ru'}
dict_of_users = requests.get(url, params=params).json()

have_sex=0
women=0
for i in range(0,500):
    sex = dict_of_users['response'][i]['sex']
    if sex==1:
        women+=1
        have_sex+=1
    if sex==2:
        have_sex+=1
women_share = round(women/have_sex,2)
print(women_share)

0.49


##### **СБОР ИНФОРМАЦИИ ИЗ ГРУПП**
***
В одном из предыдущих юнитов в качестве примера мы собрали информацию о небольшом количестве пользователей. Теперь перейдём к более реальной задаче — сбору данных о пользователях группы ВКонтакте.

Стоит отметить, что есть много сервисов, которые выгружают похожую статистику из соцсетей. Однако им свойственны недостатки универсальных решений:

* не учитываются все особенности вашего проекта;
* используется фиксированный набор метрик, дополнительную обработку данных приходится делать вам;
* не всегда бесплатны и вряд ли позволят работать с большими объёмами данных.

Теперь мы научимся считать произвольные метрики групп, собирая данные из API и работая с двумя ограничениями, которые свойственны практически всем системам:

* ограничение на количество вызовов в единицу времени;
* ограничение на количество выгружаемых строк за один запрос.

Давайте рассмотрим, как работать с этими ограничениями на примере выгрузки списка пользователей группы https://vk.com/vk социальной сети ВКонтакте.

Обратимся к [**документации**](https://vk.com/dev/groups), чтобы узнать, какие методы нам доступны для групп, — для получения списка пользователей группы доступен метод **groups.getMembers**.

Согласно документации, обязательным параметром данного метода является **group_id** — идентификатор или короткое имя группы. В нашем случае это vk: https://vk.com/vk. Протестируем, как работает метод в самом простом случае, — получим id участников группы:

In [22]:
url = 'https://api.vk.com/method/groups.getMembers' # Указываем адрес обращения
params = {'group_id': 'vk', 'v': 5.95, 'access_token': token} # Формируем строку параметров
response = requests.get(url, params = params) # Посылаем запрос
data = response.json() # Ответ сохраняем в переменной data в формате словаря
#print(data) # Выводим содержимое переменной data на экран
print(len(data['response']['items'])) # Выводим на экран количество элементов словаря

1000


Мы видим, что всего пользователей в группе больше 11 миллионов, а получили мы только первую тысячу пользователей группы. По информации, указанной в документации о параметре count, это максимум, который может отдать API за один раз.

Для получения следующей тысячи пользователей можно воспользоваться параметром **offset** (с англ. смещение), который передвинет начало отсчёта. Для выгрузки всех пользователей группы будем в цикле выгружать по 1000 пользователей (count будет всегда равен 1000), увеличивая смещение offset на величину **count**.

Для тренировки напишем цикл выгрузки первых 20 пользователей со значением count=5. Иными словами, мы будем выгружать по пять пользователей за запрос до тех пор, пока не получим информацию о 20 пользователях.

Давайте выведем на экран первые 20 пользователей из нашей первой попытки получить информацию о 1000 пользователей, чтобы мы могли сверить результат выгрузки из 20 пользователей:

In [23]:
users_for_checking = data['response']['items'][:20] # Загружаем в переменную информацию об id первых 20 пользователей в виде списка
print(users_for_checking) # Выводим перечень id первых 20 пользователей

[6, 19, 47, 54, 79, 177, 198, 212, 219, 239, 243, 345, 407, 450, 467, 485, 510, 550, 619, 640]


Теперь используем count и offset, чтобы получить те же id по пять за раз:

In [24]:
import requests # Импортируем модуль requests
token = 'dfc65ed2dfc65ed2dfc65ed21fdfba1207ddfc6dfc65ed2bda5c5957a7f5bc7684658c4' # Указываем свой сервисный токен
url = 'https://api.vk.com/method/groups.getMembers' # Указываем адрес обращения
count = 5 
offset = 0 
user_ids = [] 
max_count = 20 
while offset < max_count: 
    # Будем выгружать по count=5 пользователей, 
    # начиная с того места, где закончили на предыдущей итерации (offset) 
    print(f'Выгружаю {count} пользователей с offset = {offset}')   
    params = {'group_id': 'vk', 'v': 5.95, 'count': count, 'offset': offset, 'access_token': token} 
    response = requests.get(url, params = params) 
    data = response.json() 
    user_ids += data['response']['items'] 
    # Увеличиваем смещение на количество строк, которое мы уже выгрузили 
    offset += count 
print(user_ids) 

Выгружаю 5 пользователей с offset = 0
Выгружаю 5 пользователей с offset = 5
Выгружаю 5 пользователей с offset = 10
Выгружаю 5 пользователей с offset = 15
[6, 19, 47, 54, 79, 177, 198, 212, 219, 239, 243, 345, 407, 450, 467, 485, 510, 550, 619, 640]


In [25]:
# Сравним списки, полученные двумя способами:
print(user_ids == users_for_checking) 

True


Так как результат сравнения — True, списки идентичны. Значит, второй способ работает корректно. Теперь мы можем получить данные обо всех пользователях, выставив count = 1000 и max_count = data['response']['count'].

##### **ОГРАНИЧЕНИЕ ПО ЧАСТОТЕ ЗАПРОСОВ**
***
В API часто добавляют ограничение по частоте запросов, чтобы отдельно взятые пользователи слишком сильно не перегружали сервер. Подобное ограничение есть и у ВКонтакте — в документации указано, что можно делать не более трёх запросов в секунду.

Чтобы не следить за частотой отправки запросов с секундомером в руках, мы можем после каждого запроса делать паузу. В этом случае, даже если код будет выполняться на самом быстром компьютере, мы не нарушим установленное ограничение, так как периодичность отправки запросов будет искусственно замедлена.

Воспользуемся библиотекой **time** и методом **sleep**, с помощью которого мы можем добавить паузу, например в 0.5 секунд, после каждого запроса:

In [27]:
url = 'https://api.vk.com/method/groups.getMembers' # Указываем адрес страницы, к которой делаем запрос
count = 1000 
offset = 0  
user_ids = []  
while offset < 5000: 
    params = {'group_id': 'vk', 'v': 5.95, 'count': count, 'offset': offset, 'access_token': token} 
    response = requests.get(url, params = params) 
    data = response.json() 
    user_ids += data['response']['items'] 
    offset += count 
    print('Ожидаю 0.5 секунды...') 
    time.sleep(0.5) 
print('Цикл завершен, offset =',offset) 

Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Ожидаю 0.5 секунды...
Цикл завершен, offset = 5000


##### **ЛАЙКИ, РЕПОСТЫ И КОММЕНТАРИИ**
***
Через API новостной ленты ВКонтакте мы можем получить информацию о взаимодействии с сообщениями в ленте.

Для примера продолжим работать с группой https://vk.com/vk и рассмотрим последние 1000 сообщений в новостной ленте.

Для получения информации о сообщениях на стене в API ВКонтакте предусмотрен метод wall.get. Применим его:

In [28]:
url = 'https://api.vk.com/method/wall.get' # Указываем адрес страницы, к которой делаем запрос
params = {'domain': 'vk', 'filter': 'owner', 'count': 1000, 'offset': 0, 'access_token': token, 'v': 5.95} 
response = requests.get(url, params = params) 
pprint(response.json()) 

{'response': {'count': 484,
              'items': [{'attachments': [{'type': 'video',
                                          'video': {'access_key': '5bd99da508c4b69bd5',
                                                    'can_add': 1,
                                                    'can_add_to_faves': 1,
                                                    'can_comment': 0,
                                                    'can_like': 1,
                                                    'can_repost': 1,
                                                    'can_subscribe': 1,
                                                    'comments': 0,
                                                    'date': 1650036655,
                                                    'description': '',
                                                    'duration': 30,
                                                    'first_frame_1280': 'https://i.mycdn.me/getVideoPreview?id=2382458915369&idx

Анализируя ответ, понимаем, что по ключу count можно найти общее количество сообщений в новостной ленте, а по ключу items — сами сообщения.

In [29]:
# Посмотрим на информацию об отдельном сообщении:
response.json()['response']['items'][0] 

{'id': 1294102,
 'from_id': -22822305,
 'owner_id': -22822305,
 'date': 1650036838,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': 'Представляем мобильное приложение VK Звонков — со всеми функциями для безлимитных онлайн-конференций. Оно уже доступно в App Store (vk.cc/ccNmFQ) и Google Play (vk.cc/ccNmMx). А в браузере попробуйте веб-версию: calls.vk.com. \n\nПриложение полностью бесплатное, и все возможности VK Звонков — в быстром доступе. Подключиться к конференции по ссылке можно даже без профиля. Время разговора и количество участников не ограничены: собирайте друзей, подписчиков, слушателей онлайн-курса или всю команду большого проекта.\n\nПланируете встречи заранее? Скоро это можно будет удобно делать в VK Звонках. Можно будет создать конференцию, поставить напоминание в календаре и отправить ссылку участникам — по почте или в любом мессенджере. В нужное время звонок станет активным, и собеседники смогут присоединиться. Для регулярных встреч можно сделать звонок повторяющимс

В полях comments, likes и reposts содержится статистика по взаимодействию с сообщением пользователей (на момент получения информации) — число комментариев, лайков и репостов.
***
Давайте соберём итоговую статистику для последних десяти непустых сообщений в словарь stats. В качестве ключа будем использовать начало сообщения (если начало сообщения пустое, то информацию о таком сообщении проигнорируем), в качестве значения — список с тремя интересующими нас метриками и временем публикации (комментарии, лайки, репосты, дата публикации):

In [30]:
stats = {} 
count_post = 0 # Счётчик «непустых» сообщений
for record in response.json()['response']['items'][:]:
    title = record['text'][:30] 
    if title: 
        stats[title] = [record['comments']['count'], record['likes']['count'], record['reposts']['count'], record['date']] 
        count_post += 1 
    if count_post < 10: 
        continue 
    else: 
        break 
pprint(stats)

{'Аудитория ВКонтакте растёт: в ': [403, 1128, 268, 1648644620],
 'В VK Видео обновление для тех,': [330, 1554, 387, 1649349732],
 'В VK Видео — обновление для лю': [204, 852, 208, 1649929834],
 'В мультивселенной ВКонтакте от': [1422, 4966, 1276, 1649757486],
 'В этом месяце пользователи ВКо': [497, 1710, 393, 1648885800],
 'Весной пользователи стали созд': [155, 740, 180, 1648803000],
 'Запустили приложение VK Видео ': [547, 1415, 539, 1648119076],
 'Подсчитали: просмотры историй ': [1081, 950, 281, 1648975800],
 'Представляем мобильное приложе': [377, 865, 282, 1650036838],
 'Что посмотреть ВКонтакте? [htt': [386, 528, 90, 1648715757]}


Полный список методов ВКонтакте можно посмотреть в [**документации**](https://vk.com/dev/methods).

##### **ДРУГИЕ API**
***
Вы познакомились с интерфейсами прикладного программирования — **API** (на примере API социальной сети ВКонтакте).

API для разработчиков предоставляют и многие другие платформы. Вот список, пожалуй, самых популярных из них:

* [Google Maps API](https://developers.google.com/maps/)
* [YouTube API](https://developers.google.com/youtube/)
* [Twitter API](https://dev.twitter.com/overview/documentation)
* [Facebook API](https://developers.facebook.com/docs/)

Информацию о сторонних API можно найти в [каталоге Web API](https://www.programmableweb.com/category/all/apis). Также можно воспользоваться интернет-поиском, указав в строке поиска, например, «курсы валют API» или «прогноз погоды api», — среди первых результатов выдачи чаще всего с лёгкостью можно найти ссылки на необходимый функционал.